In [15]:
# STANDARD LIBRARY IMPORTS
import re
import csv
import os
import datetime
import subprocess
import sys # for args

# THIRD PARTY IMPORTS
import praw

# PROJECT IMPORTS
from setup import reddit
from settings import *
from tools import *
from inbox import check_messages
from reminders import send_reminders
from log_missed_comment import manually_award_comment

subreddit = reddit.subreddit(TARGET_SUBREDDIT)

# Step 1: Get the last awarded post

In [22]:
# get the last awarded post. We don't need to look back past this time.
last_award = next( reddit.user.me().comments.new() )

# Step 2: Search the most recent 1,000 comments, and record their IDs

In [57]:
num_processed_comments = 0
num_awarded_comments   = 0

# we can only go back 1,000 comments :|
for awarding_comment in subreddit.comments(limit=None):
    num_processed_comments += 1
    
    if awarding_comment.author.name != 'AutoModerator' and comment_contains_token( awarding_comment ):
        print(f'Should we award points based on comment {awarding_comment.id} by u/{awarding_comment.author.name}?')
        num_awarded_comments += 1

    if awarding_comment.created_utc <= last_award.created_utc:
        # we've gone back far enough
        print('Yay! There were less than 1,000 comments since the bot crashed.\n')
        break
        
print(f'\n\nSearched {num_processed_comments} comments. Found {num_awarded_comments} that might need awarding.')
        
print(f'There is a gap of {( awarding_comment.created_utc - last_award.created_utc ) / 60 / 60:.1f} hours between the last comment we searched, and the last award.')
print('Hopefully the above number is 0 or small!')

Should we award points based on comment ilhtgap by u/sadhak_x0?
Should we award points based on comment ilhiq96 by u/godparticle123?
Should we award points based on comment ilghg15 by u/AtmaCAM?
Should we award points based on comment ilg1ivc by u/LensSchwarzWeiss?
Should we award points based on comment ilcppo4 by u/LensSchwarzWeiss?
Should we award points based on comment ilcn36i by u/Kontact_OW?
Should we award points based on comment ilcjkue by u/SykeLopse?
Should we award points based on comment ilcal4j by u/Kontact_OW?
Should we award points based on comment ilal8k8 by u/SykeLopse?
Should we award points based on comment ilajvmr by u/Getcheb?
Should we award points based on comment ilafa9b by u/SykeLopse?
Should we award points based on comment il8clhn by u/Repartee41?
Should we award points based on comment il7ckh8 by u/SpadesD7?
Should we award points based on comment il7am26 by u/DichotomousSoul?
Should we award points based on comment il7a77a by u/DichotomousSoul?
Should we a

# Step 3: Manually award all the comments above.

Reddit will rate limit you at some point, so you'll need to wait a few mins if you have more than ~15 comments to award.

To award comments, copy the ID above into the below string, and run the cell. Repeat for all comments listed above. You may want to manually check things like:

- don't award to top level comments? 

Are these things still in `run.py`?

In [52]:
# manually_award_comment('il4m5y4')

Manually awarding points based on comment il4m5y4 by u/opekartinka.


# Step 5: Manually look for missed awards that need checking

I would do this by starting at /new, scrolling back to post from the below last-checked comment, and then manually opening all the posts after that and searching for `!CritiquePoint`. Do this until you are fairly certain you've gone far enough.

In [60]:
print('This is the last comment we were able to check:')
print(awarding_comment.permalink)

print('\nComments older than this may need checking manually.')

This is the last comment we were able to check:
/r/photocritique/comments/wt9064/thoughts_on_this_edit_didnt_have_much_lighting_at/il3iajp/

Comments older than this may need checking manually.


In [75]:
#  manually_award_comment('ikpzziq')

Manually awarding points based on comment ikpzziq by u/AK314159.
